In [1]:
# Generate Multiline Vector from Labelbox Labels

from glob import glob
import pandas as pd
import json
import rasterio as rio
from shapely.geometry import LineString
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import geopandas as gpd

# Extract Chips

import gdal
import os

# Create NDVI

import numpy as np


# Exporting Intersecting Chips

from osgeo import gdal, osr, ogr
from shutil import copyfile

In [2]:
def convert_rasters(src_dir, dest_dir, epsg_format='EPSG:3257', windows=False):
    """Converts the rasters in the src_dir into a different EPSG format,
    keeping the same folder structure and saving them in the dest_dir."""

    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    # If the output folder doesn't exist, create it
    create_dir(dest_dir)

    input_files = glob(src_dir + '/*.tif')
    # Keep track of how many files were converted
    n = 1
    total = len(input_files)
    
    for f in input_files:
        print(f'processing file {n} of {total}', end="\r", flush=True)
        
        n += 1
        
        # The way we've set it up, we save each product into a numbered folder,
        # depending on which layer it's in. To keep this structure, we need to
        # pull out the folder number from the file path.
        # How exactly to do this depends on if you're using Windows or not,
        # since the path conventions are different.
        if windows:
            filename = f.split('\\')[-1]
        else:
            filename = f.split('/')[-1]
            
        # If the respective grouping folders are not available 
        create_dir(dest_dir)
        
        output_filepath = dest_dir + filename


#         Finally, we convert
        converted = gdal.Warp(output_filepath, [f],format='GTiff',
                              dstSRS=epsg_format, resampleAlg='near')
        converted = None
        
    print('Finished')